In [1]:
#Importing libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from cuml.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from cuml import LinearRegression
from cuml.linear_model import LinearRegression

In [2]:
PATH1= '../input/ucl-ai-football-prediction/Kaggle Data/train.csv'
df= pd.read_csv(PATH1)

In [8]:
PATH2= '../input/ucl-ai-football-prediction/Kaggle Data/test.csv'
df_test= pd.read_csv(PATH2)

In [4]:
df.head()

In [5]:
df.isna()

In [6]:
sns.heatmap(df.isnull(), cbar=False)

In [9]:
df['home_adv'] = df['home_team']==df['country']
df['home_disadv'] = df['away_team']==df['country']
df_test['home_adv'] = df_test['home_team']==df_test['country']
df_test['home_disadv'] = df_test['away_team']==df_test['country']

In [10]:
teams = {}
booleanDict = {
    False : 0,
    True : 1
}
counter = 1
for team in df['home_team'] :
    if team not in teams :
        teams[team] = counter
        counter += 1
for team in df['away_team'] :
    if team not in teams :
        teams[team] = counter
        counter+=1
for i in range(len(df['home_team'])) :
    df['home_team'][i] = teams[df['home_team'][i]]
for i in range(len(df['away_team'])) :
    df['away_team'][i] = teams[df['away_team'][i]]
for i in range(len(df_test['home_team'])) :
    df_test['home_team'][i] = teams[df_test['home_team'][i]]
for i in range(len(df_test['away_team'])) :
    df_test['away_team'][i] = teams[df_test['away_team'][i]]
for i in range(len(df['neutral'])) :
    df['neutral'][i] = booleanDict[df['neutral'][i]]
    df['home_adv'][i] = booleanDict[df['home_adv'][i]]
    df['home_disadv'][i] = booleanDict[df['home_disadv'][i]]
for i in range(len(df_test['neutral'])) :
    df_test['neutral'][i] = booleanDict[df_test['neutral'][i]]
    df_test['home_adv'][i] = booleanDict[df_test['home_adv'][i]]
    df_test['home_disadv'][i] = booleanDict[df_test['home_disadv'][i]]

In [12]:
df = df.drop(['country'],axis=1)
df_test = df_test.drop(['country'],axis=1)

In [14]:
train_x,test_x,train_y,test_y = train_test_split(df.drop(['Outcome','id','home_disadv'],axis=1),df['Outcome'],test_size=0.33,random_state=0)

In [15]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(train_x, train_y)
print(train_x.columns)
clf.feature_importances_

In [24]:
y_pred_test = clf.predict(train_x)

In [25]:
accuracy_score(test_y, y_pred_test)

In [16]:
train_x

In [17]:
X = np.asarray(train_x.values, dtype=np.float32)
y = np.asarray(train_y.values, dtype=np.float32)
lr = LinearRegression(fit_intercept = False, normalize = False,algorithm = "svd")
reg = lr.fit(X,y)
preds = reg.predict(np.asarray(test_x.values, dtype=np.float32))
newPreds = []
for i in range(len(preds)) :
    if preds[i] <= 0.5 :
        newPreds.append(0)
    else :
        newPreds.append(1)
preds = reg.predict(np.asarray(test_x.values, dtype=np.float32))
from cuml.metrics.accuracy import accuracy_score
print(accuracy_score(test_y,np.asarray(newPreds)))
import cuml
MAE = cuml.metrics.regression.mean_absolute_error(test_y,np.asarray(newPreds))
print(MAE)

In [18]:
X = np.asarray(train_x.values, dtype=np.float32)
y = np.asarray(train_y.values, dtype=np.float32)
reg = LogisticRegression()
reg.fit(X,y)
preds = reg.predict(np.asarray(test_x.values, dtype=np.float32))
from cuml.metrics.accuracy import accuracy_score
print(accuracy_score(test_y,preds))

In [ ]:
#def encoder(df):
    
    #Extracting teams' names
   # H_team = np.array(df['home_team'])
   # A_team = np.array(df['away_team'])
    
    #Teams = np.hstack((H_team, A_team))
    
    #Encoding names
    #team_encoder = LabelEncoder().fit(Teams)
    
    #H_encoded = team_encoder.transform(H_team)
    #A_encoded = team_encoder.transform(A_team)

    #Creating X feature encoded values
    #H = np.expand_dims(H_encoded, axis = -1)
    #A = np.expand_dims(A_encoded, axis = -1)
    
    #X = np.concatenate([H, A], axis = 1)
    
    #Scale values
    #X = StandardScaler().fit(X).transform(X)
    #return X  

In [ ]:
#def preprocessing(df):
    
    #Removing unwanted columns:
    #X= df.drop('id', axis=1)
    
    #Extracting encoded features
   # columns = ['home_team','away_team', 'country', 'neutral']
    
    #Extracting 'Outcome' and converting to the numpy array
  #  y = np.array(df['Outcome'])
    
 #   return X, y

#X, y = preprocessing(df)

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [ ]:
#model= LogisticRegression()
#model.fit(X_train, y_train)

In [ ]:
#X= encoder(test_data)

In [ ]:
#predictions = model.predict(y_test)

In [19]:
from sklearn.metrics import classification_report

In [21]:
classification_report(test_y, preds)

In [22]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(test_y, preds))